In [3]:
from scipy import stats
from glob import glob
import re
import pandas as pd
import numpy as np
from math import atan2, degrees
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter, MaxNLocator
from scipy.interpolate import make_interp_spline
from scipy.interpolate import interp1d
from scipy import signal
import re
from pathlib import Path
from itertools import product
import os
from hampel import hampel
pd.set_option('display.max_columns', 500)

# Data Input, Parsing, and clean up

In [4]:

def combine_levels(df: pd.DataFrame) -> pd.DataFrame:
    df.columns = df.columns.map("_".join)
    return df


def read_raw_data(path: str) -> pd.DataFrame:
    raw_df = pd.read_csv(
        path,
        skiprows=[0],
        header=[0, 1, 2],
        index_col=[0],
    )

    return raw_df


def drop_likelihood(df: pd.DataFrame) -> pd.DataFrame:
    mask = df.columns.str.contains("likelihood")
    find_filter = df.columns[mask]
    filter_df = df.drop(columns=find_filter)
    return filter_df

def fix_flip(df):
    boundary = 700
    problem_columns = [
        ("MouseTopRight_Snout", "MouseBottomLeft_Snout"),
        ("MouseTopRight_Forelimb", "MouseBottomLeft_Forelimb"),
        ("MouseTopRight_Wrist", "MouseBottomLeft_Wrist"),
        ("MouseTopRight_Elbow", "MouseBottomLeft_Elbow"),
        ("MouseTopRight_LowerShoulder", "MouseBottomLeft_LowerShoulder"),
        ("MouseTopRight_UpperShoulder", "MouseBottomLeft_UpperShoulder"),
        ("MouseTopRight_IlliacCrest", "MouseBottomLeft_IlliacCrest"),
        ("MouseTopRight_Hip", "MouseBottomLeft_Hip"),
        ("MouseTopRight_Knee", "MouseBottomLeft_Knee"),
        ("MouseTopRight_Ankle", "MouseBottomLeft_Ankle"),
        ("MouseTopRight_Hindlimb", "MouseBottomLeft_Hindlimb"),
        ("MouseTopRight_TailBase", "MouseBottomLeft_TailBase"),
        ("MouseTopRight_TailCenter", "MouseBottomLeft_TailCenter"),
        ("MouseTopRight_TailTip", "MouseBottomLeft_TailTip"),
    ]
    for index, row in df.iterrows():
        for side_pair in problem_columns:
            top_R_y_value, bottom_L_y_value = row[f'{side_pair[0]}_y'], row[f'{side_pair[1]}_y']
            top_R_x_value, bottom_L_x_value = row[f'{side_pair[0]}_x'], row[f'{side_pair[1]}_x']
            if top_R_y_value >= boundary and bottom_L_y_value < boundary:
                row[f'{side_pair[0]}_y'], row[f'{side_pair[1]}_y'] = bottom_L_y_value, top_R_y_value
                row[f'{side_pair[0]}_x'], row[f'{side_pair[1]}_x'] = bottom_L_x_value, top_R_x_value
    return df

def clean_data(raw_df: pd.DataFrame) -> pd.DataFrame:
    clean_df = (
        raw_df.rename_axis(index="frame")
        .interpolate(method="linear")
        .pipe(combine_levels)
        .pipe(drop_likelihood)
        .pipe(fix_flip)
    )
    return clean_df

def side_outlier_detection(df):
    indices = set()
    columns_to_exclude = df.columns[-16:]
    for column in df.columns:
        if column not in columns_to_exclude:
            outlier_index = hampel(df[column], window_size=10, n=100)
            indices.update(outlier_index)
    drop_df = df.drop(list(indices))
    return drop_df

def zscore_outlier_detection(df):
    zscore_df = df.copy()  
    columns_to_check = df.columns[-16:]
    threshold = 3
    for column in columns_to_check:
        if "_y" in column:
            zscores = stats.zscore(df[column])
            outliers = np.abs(zscores) > threshold
            zscore_df.loc[outliers, column] = np.nan
    clean_zdf = zscore_df.dropna()
    return clean_zdf

def outlier_clean(df):
    clean_df = df.pipe(side_outlier_detection).pipe(zscore_outlier_detection)
    return clean_df

def walking_fig(df):
    fig = plt.figure(figsize=(10, 8))
    plt.rcParams["axes.linewidth"] = 3
    
    final_time = (df.index[-1]) / 200  
    plt.xlim(0.0, final_time)
    plt.tick_params(axis='x', labelsize=30, length=9, width=3)
    plt.tick_params(axis='y', labelsize=30, length=9, width=3)
    
    # Labels
    plt.title(f"{column_name} Oscillation Pattern", fontsize='20')
    plt.xlabel("Time (In sec)", fontsize='30')
    plt.ylabel("Distance Traveled (cm)", fontsize='30')
    plt.legend(["Left Hindpaw", "Left Forepaw", "Right Hindpaw", "Right Forepaw"], fontsize="10", loc='lower right')

    # Scatter plot 
    plt.plot((df.index)/200, df["MouseBottomLeft_Hindlimb_x"]/24, color="darkslategray", linestyle="dashed", linewidth=4)
    plt.plot((df.index)/200, df["MouseBottomLeft_Forelimb_x"]/24, color='thistle', linewidth=4)
    plt.plot((df.index)/200, df["MouseTopRight_Hindlimb_x"]/24, color='lightcoral', linestyle="dashed", linewidth=4)
    plt.plot((df.index)/200, df["MouseTopRight_Forelimb_x"]/24, color='aquamarine', linewidth=4)
    
    # Save
    plt.savefig(f"{column_name} Oscillation Pattern")

# Combination Lists

In [3]:
# This creates a list of all possible body parts combinations. Can change in the consolidate and output functions

import itertools
#All body parts
Top_Right = ['MouseTopRight_Snout', 'MouseTopRight_Forelimb', 'MouseTopRight_Wrist', 'MouseTopRight_Elbow', 'MouseTopRight_LowerShoulder', 'MouseTopRight_UpperShoulder', 'MouseTopRight_IlliacCrest', 'MouseTopRight_Hip', 'MouseTopRight_Knee', 'MouseTopRight_Ankle', 'MouseTopRight_Hindlimb', 'MouseTopRight_TailBase', 'MouseTopRight_TailCenter', 'MouseTopRight_TailTip']
Center = ['single_MouseCenter_Snout', 'single_MouseCenter_LeftForelimb', 'single_MouseCenter_RightForelimb', 'single_MouseCenter_LeftHindlimb', 'single_MouseCenter_RightHindlimb', 'single_MouseCenter_TailBase', 'single_MouseCenter_TailCenter', 'single_MouseCenter_TailTip']
Bottom_Left = ['MouseBottomLeft_Snout', 'MouseBottomLeft_Forelimb', 'MouseBottomLeft_Wrist', 'MouseBottomLeft_Elbow', 'MouseBottomLeft_LowerShoulder', 'MouseBottomLeft_UpperShoulder', 'MouseBottomLeft_IlliacCrest', 'MouseBottomLeft_Hip', 'MouseBottomLeft_Knee', 'MouseBottomLeft_Ankle', 'MouseBottomLeft_Hindlimb', 'MouseBottomLeft_TailBase', 'MouseBottomLeft_TailCenter', 'MouseBottomLeft_TailTip']

#All possible combos (2 for distances, 3 for angles)
TR_Distcomb = list(itertools.combinations(Top_Right, 2))
TR_Anglecomb = list(itertools.permutations(Top_Right, 3))

C_Distcomb = list(itertools.combinations(Center, 2))
C_Anglecomb = list(itertools.permutations(Center, 3))

BL_Distcomb = list(itertools.combinations(Bottom_Left, 2))
BL_Anglecomb = list(itertools.permutations(Bottom_Left, 3))

All_Dists = TR_Distcomb + C_Distcomb + BL_Distcomb
All_Angles = TR_Anglecomb + C_Anglecomb + BL_Anglecomb

# Calculate Distances of Interest

In [5]:
dist_combo_side = [
    ("MouseTopRight_Forelimb", "MouseTopRight_Wrist"),
    ("MouseTopRight_Wrist", "MouseTopRight_Elbow"),
    ("MouseTopRight_Elbow", "MouseTopRight_LowerShoulder"),
    ("MouseTopRight_LowerShoulder", "MouseTopRight_UpperShoulder"),
    ("MouseTopRight_UpperShoulder", "MouseTopRight_IlliacCrest"),
    ("MouseTopRight_IlliacCrest", "MouseTopRight_Hip"),
    ("MouseTopRight_Hip", "MouseTopRight_Knee"),
    ("MouseTopRight_Knee", "MouseTopRight_Ankle"),
    ("MouseTopRight_Ankle", "MouseTopRight_Hindlimb"),
    ("MouseTopRight_TailBase", "MouseTopRight_TailCenter"),
    ("MouseTopRight_TailCenter", "MouseTopRight_TailTip"),
    ("MouseBottomLeft_Forelimb", "MouseBottomLeft_Wrist"),
    ("MouseBottomLeft_Wrist", "MouseBottomLeft_Elbow"),
    ("MouseBottomLeft_Elbow", "MouseBottomLeft_LowerShoulder"),
    ("MouseBottomLeft_LowerShoulder", "MouseBottomLeft_UpperShoulder"),
    ("MouseBottomLeft_UpperShoulder", "MouseBottomLeft_IlliacCrest"),
    ("MouseBottomLeft_IlliacCrest", "MouseBottomLeft_Hip"),
    ("MouseBottomLeft_IlliacCrest", "MouseBottomLeft_Hindlimb"),
    ("MouseBottomLeft_Hip", "MouseBottomLeft_Knee"),
    ("MouseBottomLeft_Knee", "MouseBottomLeft_Ankle"),
    ("MouseBottomLeft_Ankle", "MouseBottomLeft_Hindlimb"),
    ("MouseBottomLeft_Forelimb", "MouseBottomLeft_Hindlimb"),
    ("MouseBottomLeft_Knee", "MouseBottomLeft_Hindlimb"),
    ("MouseBottomLeft_TailBase", "MouseBottomLeft_TailCenter"),
    ("MouseBottomLeft_TailCenter", "MouseBottomLeft_TailTip"),
]
dist_combo_center = [
    ("single_MouseCenter_LeftForelimb", "single_MouseCenter_RightForelimb"),
    ("single_MouseCenter_LeftHindlimb", "single_MouseCenter_RightHindlimb"),
    ("single_MouseCenter_LeftForelimb", "single_MouseCenter_LeftHindlimb"),
    ("single_MouseCenter_RightForelimb", "single_MouseCenter_RightHindlimb"),
    ("single_MouseCenter_LeftHindlimb", "single_MouseCenter_TailBase"),
    ("single_MouseCenter_RightHindlimb", "single_MouseCenter_TailBase"),
    ("single_MouseCenter_TailBase", "single_MouseCenter_TailCenter"),
    ("single_MouseCenter_TailCenter", "single_MouseCenter_TailTip"),
]
relevant_dist_side = [
    ("MouseTopRight_LowerShoulder", "MouseTopRight_Snout"),
    ("MouseTopRight_IlliacCrest", "MouseTopRight_Hindlimb"),
    ("MouseTopRight_IlliacCrest", "MouseTopRight_UpperShoulder"),
    ("MouseBottomLeft_LowerShoulder", "MouseBottomLeft_Snout"),
    ("MouseBottomLeft_IlliacCrest", "MouseBottomLeft_Hindlimb"),
    ("MouseBottomLeft_IlliacCrest", "MouseBottomLeft_UpperShoulder"),
]
relevant_dist_center = [
    ("single_MouseCenter_LeftForelimb", "single_MouseCenter_RightForelimb"),
    ("single_MouseCenter_LeftHindlimb", "single_MouseCenter_RightHindlimb"),
    ("single_MouseCenter_LeftForelimb", "single_MouseCenter_LeftHindlimb"),
    ("single_MouseCenter_RightForelimb", "single_MouseCenter_RightHindlimb"),
    ("single_MouseCenter_LeftHindlimb", "single_MouseCenter_TailBase"),
    ("single_MouseCenter_RightHindlimb", "single_MouseCenter_TailBase"),
]

def distance_center(x1, y1, x2, y2): 
    pixel_distance = np.sqrt(
        np.square(x2 - x1) + np.square(y2 - y1)
    )
    cm_distance = pixel_distance / 33 # Pixels per cm for center view
    return cm_distance

def get_dist_center(df, bp_map):
    kwargs = {}
    for pair in bp_map: 
        coord1, coord2 = pair 
        coord1_x = f"{coord1}_x"
        coord1_y = f"{coord1}_y"
        coord2_x = f"{coord2}_x"
        coord2_y = f"{coord2}_y"
    
        kwargs[f"{coord1}_{coord2}_dist"] = distance_center(
        df[coord1_x],
        df[coord1_y],
        df[coord2_x],
        df[coord2_y],
        )
    new_df = pd.DataFrame()
    return new_df.assign(**kwargs)
 
def distance_side(x1, y1, x2, y2): 
    pixel_distance = np.sqrt(
        np.square(x2 - x1) + np.square(y2 - y1)
    )
    cm_distance = pixel_distance / 24 # Pixels per cm for side view
    return cm_distance

def get_dist_side(df, bp_map):
    kwargs = {}
    for pair in bp_map: 
        coord1, coord2 = pair 
        coord1_x = f"{coord1}_x"
        coord1_y = f"{coord1}_y"
        coord2_x = f"{coord2}_x"
        coord2_y = f"{coord2}_y"
    
        kwargs[f"{coord1}_{coord2}_dist"] = distance_side(
        df[coord1_x],
        df[coord1_y],
        df[coord2_x],
        df[coord2_y],
        )
    new_df = pd.DataFrame()
    return new_df.assign(**kwargs) 


# Calculate Angles of Interest

In [6]:
angle_combo = [
    (
        "MouseTopRight_Forelimb",
        "MouseTopRight_Wrist",
        "MouseTopRight_Elbow",
    ),
    (
        "MouseTopRight_Wrist",
        "MouseTopRight_Elbow",
        "MouseTopRight_LowerShoulder",
    ),
    (
        "MouseTopRight_Elbow",
        "MouseTopRight_LowerShoulder",
        "MouseTopRight_UpperShoulder",
    ),
    (
        "MouseTopRight_LowerShoulder",
        "MouseTopRight_UpperShoulder",
        "MouseTopRight_IlliacCrest",
    ),
    (
        "MouseTopRight_UpperShoulder",
        "MouseTopRight_IlliacCrest",
        "MouseTopRight_Hip",
    ),
    (
        "MouseTopRight_IlliacCrest",
        "MouseTopRight_Hip",
        "MouseTopRight_Knee",
    ),
    (
        "MouseTopRight_Hip",
        "MouseTopRight_Knee",
        "MouseTopRight_Ankle",
    ),
    (
        "MouseTopRight_Knee",
        "MouseTopRight_Ankle",
        "MouseTopRight_Hindlimb",
    ),
    (
        "MouseTopRight_TailBase",
        "MouseTopRight_TailCenter",
        "MouseTopRight_TailTip",
    ),
    (
        "MouseBottomLeft_Forelimb",
        "MouseBottomLeft_Wrist",
        "MouseBottomLeft_Elbow",
    ),
    (
        "MouseBottomLeft_Wrist",
        "MouseBottomLeft_Elbow",
        "MouseBottomLeft_LowerShoulder",
    ),
    (
        "MouseBottomLeft_Elbow",
        "MouseBottomLeft_LowerShoulder",
        "MouseBottomLeft_UpperShoulder",
    ),
    (
        "MouseBottomLeft_LowerShoulder",
        "MouseBottomLeft_UpperShoulder",
        "MouseBottomLeft_IlliacCrest",
    ),
    (
        "MouseBottomLeft_UpperShoulder",
        "MouseBottomLeft_IlliacCrest",
        "MouseBottomLeft_Hip",
    ),
    (
        "MouseBottomLeft_IlliacCrest",
        "MouseBottomLeft_Hip",
        "MouseBottomLeft_Knee",
    ),
    (
        "MouseBottomLeft_Hip",
        "MouseBottomLeft_Knee",
        "MouseBottomLeft_Ankle",
    ),
    (
        "MouseBottomLeft_Knee",
        "MouseBottomLeft_Ankle",
        "MouseBottomLeft_Hindlimb",
    ),
    (
        "MouseBottomLeft_TailBase",
        "MouseBottomLeft_TailCenter",
        "MouseBottomLeft_TailTip",
    ),
    (
        "single_MouseCenter_LeftHindlimb",
        "single_MouseCenter_TailBase",
        "single_MouseCenter_RightHindlimb",
    ),
]

def angle_between(row: pd.Series):
    x1 = row.iloc[0]
    y1 = row.iloc[1]
    x2 = row.iloc[2]
    y2 = row.iloc[3]
    x3 = row.iloc[4]
    y3 = row.iloc[5]
    
    deg1 = (
        360 + degrees(atan2(x3 - x2, y3 - y2))
    ) % 360
    deg2 = (
        360 + degrees(atan2(x1 - x2, y1 - y2))
    ) % 360
    angle =  deg2 - deg1 if deg1 <= deg2 else 360 - (deg1 - deg2)
    if angle > 180:
        angle = 360 - angle
    return angle


def get_angles(df, bp_map):
    dict = {}
    for pair in bp_map:
        coord1, coord2, coord3 = pair
        coord1_x = f'{coord1}_x'
        coord1_y = f'{coord1}_y'
        coord2_x = f'{coord2}_x'
        coord2_y = f'{coord2}_y'
        coord3_x = f'{coord3}_x'
        coord3_y = f'{coord3}_y'

        col_list = [
            coord1_x,
            coord1_y,
            coord2_x,
            coord2_y,
            coord3_x,
            coord3_y,
        ]
        dict[f"{coord2}_angle"] = df[col_list].apply(angle_between, axis=1)
    new_df = pd.DataFrame()
    return new_df.assign(**dict)

# Calculate Speed and Acceleration For Each Limb

In [20]:
body_parts = [
    'MouseTopRight_Snout', 'MouseTopRight_Forelimb', 'MouseTopRight_Wrist',
    'MouseTopRight_Elbow', 'MouseTopRight_LowerShoulder', 'MouseTopRight_UpperShoulder',
    'MouseTopRight_IlliacCrest', 'MouseTopRight_Hip', 'MouseTopRight_Knee',
    'MouseTopRight_Ankle', 'MouseTopRight_Hindlimb', 'MouseTopRight_TailBase',
    'MouseTopRight_TailCenter', 'MouseTopRight_TailTip', 'MouseBottomLeft_Snout',
    'MouseBottomLeft_Forelimb', 'MouseBottomLeft_Wrist', 'MouseBottomLeft_Elbow',
    'MouseBottomLeft_LowerShoulder', 'MouseBottomLeft_UpperShoulder', 'MouseBottomLeft_IlliacCrest',
    'MouseBottomLeft_Hip', 'MouseBottomLeft_Knee', 'MouseBottomLeft_Ankle',
    'MouseBottomLeft_Hindlimb', 'MouseBottomLeft_TailBase', 'MouseBottomLeft_TailCenter',
    'MouseBottomLeft_TailTip', 'single_MouseCenter_Snout', 'single_MouseCenter_LeftForelimb',
    'single_MouseCenter_RightForelimb', 'single_MouseCenter_LeftHindlimb', 'single_MouseCenter_RightHindlimb',
    'single_MouseCenter_TailBase', 'single_MouseCenter_TailCenter', 'single_MouseCenter_TailTip'
]
relevant_bp_side = [
    'MouseTopRight_Snout', 'MouseTopRight_Forelimb','MouseTopRight_Hindlimb', 'MouseTopRight_TailBase', 
    'MouseBottomLeft_Snout','MouseBottomLeft_Forelimb','MouseBottomLeft_Hindlimb', 
    'MouseBottomLeft_TailBase'
]

def calc_speed_acc(df, body_parts, fps=200):
    results = {}
    Time_fps = 1 / fps
    for bp in body_parts:
        x, y = f"{bp}_x", f"{bp}_y"
        new_rows = (
            df.assign(
                new_frame_x=lambda df: df[x].shift(-1),
                new_frame_y=lambda df: df[y].shift(-1),
                frame=np.arange(1, len(df.index) + 1),
            ).dropna().assign(
                Cummulative_Time=lambda df: df["frame"] / fps,
                Distance=lambda df: distance_side(
                    df[x],
                    df[y],
                    df["new_frame_x"],
                    df["new_frame_y"],
                ), 
                Cumulative_Distance=lambda df: df["Distance"].cumsum(),
                Speed=lambda df: df["Distance"] / Time_fps,
                Velocity=lambda df: ((df["new_frame_x"] - df[x])/24) / Time_fps,####velocity for x only in incorporate position
                shift_Velocity=lambda df: df[["Velocity"]].shift(-1),
                delta_velocity=lambda df: (df["Velocity"] - df["shift_Velocity"]),
                Acceleration=lambda df: (df["delta_velocity"]) / Time_fps,
            ))
        
        # Set boundary for max values
        new_rows.loc[new_rows['Speed'] > 100, 'Speed'] = None
        
        results[bp] = new_rows[["Cumulative_Distance", "Velocity", "Speed", "Acceleration"]]
    df_list = []
    for bp, rows in results.items():
        rows.columns = [f"{bp}_{col}" for col in rows.columns]
        df_list.append(rows)
    s_a_df = pd.concat(df_list, axis=1)
    return s_a_df

# Calculate Heights from Base Height

In [21]:
limb_heights = [
    ("MouseBottomLeft_Hindlimb", "MouseBottomLeft_TailTip"),
    ("MouseBottomLeft_Hindlimb", "MouseBottomLeft_TailBase"),
    ("MouseBottomLeft_Hindlimb", "MouseBottomLeft_Hindlimb"),
    ("MouseBottomLeft_Hindlimb", "MouseBottomLeft_IlliacCrest"),
    ("MouseBottomLeft_Hindlimb", "MouseBottomLeft_UpperShoulder"),
    ("MouseBottomLeft_Hindlimb", "MouseBottomLeft_Forelimb"),
    ("MouseBottomLeft_Hindlimb", "MouseBottomLeft_Snout"),
    ("MouseTopRight_Hindlimb", "MouseTopRight_TailTip"),
    ("MouseTopRight_Hindlimb", "MouseTopRight_TailBase"),
    ("MouseTopRight_Hindlimb", "MouseTopRight_Hindlimb"),
    ("MouseTopRight_Hindlimb", "MouseTopRight_IlliacCrest"),
    ("MouseTopRight_Hindlimb", "MouseTopRight_UpperShoulder"),
    ("MouseTopRight_Hindlimb", "MouseTopRight_Forelimb"),
    ("MouseTopRight_Hindlimb", "MouseTopRight_Snout"),
]
relevant_heights = [
    ("MouseBottomLeft_Hindlimb", "MouseBottomLeft_TailTip"),
    ("MouseBottomLeft_Hindlimb", "MouseBottomLeft_TailBase"),
    ("MouseBottomLeft_Hindlimb", "MouseBottomLeft_Hindlimb"),
    ("MouseBottomLeft_Hindlimb", "MouseBottomLeft_Forelimb"),
    ("MouseBottomLeft_Hindlimb", "MouseBottomLeft_IlliacCrest"),
    ("MouseTopRight_Hindlimb", "MouseTopRight_TailTip"),
    ("MouseTopRight_Hindlimb", "MouseTopRight_TailBase"),
    ("MouseTopRight_Hindlimb", "MouseTopRight_Hindlimb"),
    ("MouseTopRight_Hindlimb", "MouseTopRight_Forelimb"),
    ("MouseTopRight_Hindlimb", "MouseTopRight_IlliacCrest"),
]

def height(df: pd.DataFrame, bp_map) -> pd.DataFrame:
    lengths = {}
    for pair in bp_map:
        constant, bp = pair
        constant_y = f"{constant}_y"
        bp_y = f"{bp}_y"
        if "MouseBottomLeft_" in constant:
            lengths[f"{bp}_height"] = abs(df[bp_y] - df[constant_y].max()) / 24  #convert to cm from side profile
        elif "MouseTopRight_" in constant:
            lengths[f"{bp}_height"] = abs(df[bp_y] - df[constant_y].min()) / 24  #convert to cm from side profile
    height_df = pd.DataFrame()
    return height_df.assign(**lengths)

# Locate Data and Collect File Names

In [ ]:
# Run this cell first to have the files loaded
import os
import re

def extract_dpi(filename):
    match = re.search(r'(\d+)dpi', filename)
    if match:
        return int(match.group(1))
    return -1 

def get_csv_filenames(folder_path):
    csv_filenames = []
    for filename in os.listdir(folder_path):
        if filename.endswith("filtered.csv"):
            csv_filenames.append(os.path.join(folder_path, filename))
    return csv_filenames

# Replace 'folder_path' with the actual path of your folder containing CSV files
folder_path =  'Folder_Path'
csv_files = get_csv_filenames(folder_path)

# Sort the filenames based on dpi number
csv_files_sorted = sorted(csv_files, key=lambda x: extract_dpi(x))
files = []
print("CSV files in the folder (sorted by dpi number) with full path:")
for file in csv_files_sorted:
    files.append(file)
display(files)

# Consolidate and Output

In [ ]:
DataFs = []
File_list = files

last_df = pd.DataFrame()

for file in File_list:
    match = re.search(r"\\([^\\]+)DLC", file)
    if match:
        column_name = match.group(1)
    # print(column_name)
    df = read_raw_data(file)
    ccdf = clean_data(df)
    cdf = outlier_clean(ccdf)
    walking_fig(cdf)
    ddf1 = get_dist_side(cdf, relevant_dist_side) # Can change to All side distance combinations
    ddf2 = get_dist_center(cdf, relevant_dist_center)# Can change to All center distance combinations
    ddf = ddf1.join(ddf2, on='frame')
    adf = get_angles(cdf, angle_combo)
    sav_df = calc_speed_acc(cdf, relevant_bp_side, fps=200)
    hdf = height(cdf, relevant_heights)
    
    ddf_cols = [col for col in ddf.columns if 'dist' in col]
    adf_cols = [col for col in adf.columns if 'angle' in col]
    sav_df_cols = [col for col in sav_df.columns if 'Speed' in col or 'Acceleration' in col or 'Velocity' in col]
    hdf_cols = [col for col in hdf.columns if 'height' in col]
    
    stats_dict = {}
    
    # Calculate descriptive statistics for distances
    for col in ddf_cols:
        col_mean = f"{col}_mean"
        col_median = f"{col}_median"
        col_min = f"{col}_min"
        col_max = f"{col}_max"
        col_std = f"{col}_std"
        
        stats_dict[col_mean] = ddf[col].mean()
        stats_dict[col_median] = ddf[col].median()
        stats_dict[col_min] = ddf[col].min()
        stats_dict[col_max] = ddf[col].max()
        stats_dict[col_std] = ddf[col].std()
    
    # Calculate descriptive statistics for angles
    for col in adf_cols:
        col_mean = f"{col}_mean"
        col_median = f"{col}_median"
        col_min = f"{col}_min"
        col_max = f"{col}_max"
        col_std = f"{col}_std"
        
        stats_dict[col_mean] = adf[col].mean()
        stats_dict[col_median] = adf[col].median()
        stats_dict[col_min] = adf[col].min()
        stats_dict[col_max] = adf[col].max()
        stats_dict[col_std] = adf[col].std()
    
    for col in sav_df_cols:
        col_mean = f"{col}_mean"
        col_median = f"{col}_median"
        col_min = f"{col}_min"
        col_max = f"{col}_max"
        col_std = f"{col}_std"

        stats_dict[col_mean] = sav_df[col].mean()
        stats_dict[col_median] = sav_df[col].median()
        stats_dict[col_min] = sav_df[col].min()
        stats_dict[col_max] = sav_df[col].max()
        stats_dict[col_std] = sav_df[col].std()

    
    # Calculate descriptive statistics for heights
    for col in hdf_cols:
        col_mean = f"{col}_mean"
        col_median = f"{col}_median"
        col_min = f"{col}_min"
        col_max = f"{col}_max"
        col_std = f"{col}_std"
        
        stats_dict[col_mean] = hdf[col].mean()
        stats_dict[col_median] = hdf[col].median()
        stats_dict[col_min] = hdf[col].min()
        stats_dict[col_max] = hdf[col].max()
        stats_dict[col_std] = hdf[col].std()
    
    stats_df = pd.DataFrame(stats_dict, index=[column_name])
    
    last_df = pd.concat([last_df, stats_df])

# Transpose the last_df DataFrame
last_df = last_df.T
last_df.to_excel('Name of your File.xlsx', index=True)

# Average Columns Based on Names

In [ ]:
# To average multiple runs on the MR
similar_columns = {}
for col in last_df.columns:
    prefix = '_'.join(filter(None, col.split('_')[:-1]))
    if prefix not in similar_columns:
        similar_columns[prefix] = []
    similar_columns[prefix].append(col)

averaged_df = pd.DataFrame()
for prefix, cols in similar_columns.items():
    averaged_df[prefix] = last_df[cols].mean(axis=1)
averaged_df.to_excel('Name of your File Averaged.xlsx', index=True)